In [3]:
from dotenv import load_dotenv
import os
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import OllamaEmbeddings

In [4]:
load_dotenv()
model_name = "llama2"
model = Ollama(model=model_name)
embeddings = OllamaEmbeddings(model=model_name)

parser = StrOutputParser()  # Output parser for the model
chain = model | parser  # Chain the model and the parser
# chain.invoke(
#     "Tell me a joke"
# )  # Invoke the chain with the input and returns the string output

C:\Users\Karan Shah\AppData\Local\Temp\ipykernel_19812\2757306417.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model=model_name)
C:\Users\Karan Shah\AppData\Local\Temp\ipykernel_19812\2757306417.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=model_name)


In [5]:
template = """
Answer the given question based on context below.
If you don't know the answer, you can simply say "I don't know the answer".

context: {context}

question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="here is context", question="here is question")


chain = prompt | model | parser
chain.invoke(
    {"context": "the name of the person is John", "question": "what's my name?"}
)

'I\'m happy to help! Based on the context you provided, the answer to "what\'s my name?" is "John."'

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("mlschool.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'mlschool.pdf', 'page': 0}, page_content='Building Machine Learning Systems That Don\'t\nSuck\nA live, interactive program that\'ll help you build production-readymachine\nlearning systems from the ground up.\nNext cohort:\xa0November5 - 22, 2024\nCheck the schedulefor more details about upcoming cohorts.\nI want to join!Sign in\nLearn how to design, build, deploy, and scale machine learning\nsystems to solve real-world problems.\nI\'ll lose my mind if I see another book or course teaching people the same basic\nideas for the hundredth time. Most people are stuck in beginner mode, and finding\nhelp to solve real-world problems is hard.\nI want to change that.\nI started writing software 30 years ago. I\'ve written pipelines and trained models\nfor some of the largest companies in the world. I want to show you how to do the\nsame.\nThis is the class I wish I had taken when I started."This is the best machine learning course I\'ve done.\nWorth every cent."\n

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vector_store = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

c:\Users\Karan Shah\Desktop\chatbot\venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [8]:
retriver = vector_store.as_retriever()

retriver.invoke("Machine Learning")

[Document(metadata={'source': 'mlschool.pdf', 'page': 3}, page_content="You'll learn best practices to tackle the most significant challenges machine\nlearning engineers face to build, evaluate, run, monitor, and maintain\nmachine learning systems in real-world scenarios.\nYou'll learn how to use techniques like active learning, distributed training,\nadversarial validation, human-in-the-loop deployments, model compression,\ntest-time augmentation, testing in production, among many others.\nYou'll learn how to create training, deploying, monitoring, and inference\npipelines using Amazon SageMaker and open-source tools.\nForget about theoretical concepts. This program will show you some of the things\nI've learned from real-life examples I've built during more than 30 years in the\nindustry.\nCheck the program syllabus\xa0→\nReal-life examples and case studies\nLearn from practical experience building machine\nlearning systems that work in the real world.\nLive, interactive sessions\nAs

In [9]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriver,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [10]:
questions = [
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How many coding assignments are there in the program?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of the course?
Answer:  Based on the context provided, the purpose of the course is to teach students how to build machine learning systems that work in the real world. The course aims to provide practical, hands-on training for technical professionals who are ready to put in the work and learn from years of experience and real-world examples. The program will help students unlearn what they think machine learning is and learn best practices to tackle the most significant challenges machine learning engineers face when building, evaluating, running, monitoring, and maintaining machine learning systems in real-world scenarios.

Question: How many hours of live sessions?
Answer:  Based on the context provided, the answer to the question "How many hours of live sessions?" is 18 hours.

Question: How many coding assignments are there in the program?
Answer:  Based on the context provided, there are 100 coding assignments in the program.



In [11]:
question = input("Enter your question: ")
print(f"Question: {question}")
print(f"Answer: {chain.invoke({'question': question})}")

Answer: Based on the provided context, it appears that the course is focused on teaching students how to build practical machine learning systems that work in the real world. The course covers a range of topics, including data cleaning and feature engineering, building models, and deploying and monitoring machine learning systems. The course also provides hands-on training through live sessions and code walkthroughs, and offers lifetime access to past and future cohorts.
